In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import time
import matplotlib.pyplot as plt

In [2]:
mnist = input_data.read_data_sets('./data/mnist',one_hot=True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


In [17]:
learning_rate = 1e-4
n_iter = 5000
batch_size = 50

In [4]:
def conv2d(X,w):
  return tf.nn.conv2d(X, w, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(X,name):
  return tf.nn.max_pool(X, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

In [19]:
X = tf.placeholder(tf.float32,shape=[None,784],name="X")
Y = tf.placeholder(tf.float32,shape=[None,10],name="Y")

In [6]:
#first convolutional layer: ip->conv->relu->pool with 32 filters of size 5*5
#o/p dims: N*14*14*32
w1 = tf.Variable(tf.truncated_normal(shape=[5,5,1,32],stddev=0.1),tf.float32,name="w1") #w1 dims: F*F*C*K 
b1 = tf.Variable(tf.constant(0.1,shape=[32]),tf.float32,name="b1") #b1 dims: K

In [7]:
h_conv1 = tf.nn.relu(conv2d(tf.reshape(X,[-1,28,28,1]),w1) + b1,name="h_conv1") 
h_pool1 = max_pool_2x2(h_conv1,name="h_pool1")

In [8]:
#second convolutional layer with 64 filters of size 5*5: ip->conv->relu->pool
#o/p dims: N*7*7*64
w2 = tf.Variable(tf.truncated_normal(shape=[5,5,32,64],stddev=0.1),tf.float32,name="w2")
b2 = tf.Variable(tf.constant(0.1,shape=[64]),tf.float32,name="b2")

In [9]:
h_conv2 = tf.nn.relu(conv2d(h_pool1,w2) + b2,name="h_conv2")
h_pool2 = max_pool_2x2(h_conv2,name="h_pool2")

In [10]:
#densely connected layer of 1024 neurons: affine->relu
#o/p dims: N*1024
w3 = tf.Variable(tf.truncated_normal(shape=[7*7*64,1024],stddev=0.1),tf.float32,name="w3")
b3 = tf.Variable(tf.constant(0.1,shape=[1024]),tf.float32,name="b3")

In [11]:
h_fc1 = tf.nn.relu(tf.matmul(tf.reshape(h_pool2,[-1,7*7*64]),w3) + b3,name="h_fc1")

In [12]:
#dropout to reduce overfitting. significant improvement for large networks
prob = tf.placeholder(tf.float32) #probability of neuron being on
h_fc1_dropout = tf.nn.dropout(h_fc1,prob)

In [13]:
#softmax layer for classification
#o/p dims: N*10
w4 = tf.Variable(tf.truncated_normal(shape=[1024,10],stddev=0.1),tf.float32,name="w4")
b4 = tf.Variable(tf.constant(0.1,shape=[10]),tf.float32,name="b4")

In [14]:
scores = tf.add(tf.matmul(h_fc1_dropout,w4),b4,name="scores")
entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=scores,name="entropy")
loss = tf.reduce_mean(entropy,name="loss")
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [15]:
correct_pred = tf.equal(tf.argmax(scores,1),tf.argmax(Y,1),name="correct_pred")
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32),name="accuracy")

In [20]:
#run session: train and test model
with tf.Session() as sess:
    #writer = tf.summary.FileWriter('./graphs/CNN',sess.graph)
    
    sess.run(tf.global_variables_initializer())
    
    for i in range(n_iter):
        X_batch,Y_batch = mnist.train.next_batch(batch_size)
        _,l,acc = sess.run([optimizer,loss,accuracy],feed_dict={X:X_batch,Y:Y_batch,prob:0.5})
        if i % 100 == 0:
            print "Iter: ",i,", loss: ",l,", training accuracy: ",acc
    
    test_acc = sess.run([accuracy],feed_dict={X:mnist.test.images,Y:mnist.test.labels,prob:1.0})
    #writer.close()

InvalidArgumentError: You must feed a value for placeholder tensor 'X' with dtype float and shape [50,784]
	 [[Node: X = Placeholder[dtype=DT_FLOAT, shape=[50,784], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'X', defined at:
  File "/Users/prerna135/anaconda/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/prerna135/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/prerna135/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/prerna135/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/prerna135/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/Users/prerna135/anaconda/lib/python2.7/site-packages/zmq/eventloop/ioloop.py",